In [ ]:
import numpy as np
import torch

from matplotlib import pyplot as plt
from torch import optim

from models_code.experiments import test_eval
from models_code.experiments import correlation_test_error_uncertainty
from models_code.experiments import non_distribution
from models_code.experiments import softmax2d
from models_code.sentiment import generator_out_of_matrix
from models_code.sentiment import Movie
from models_code.sentiment import ISMovie
from models_code.sentiment import perform_training_sentiment
from models_code.utilities import load_model
from models_code.utilities import create_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

In [ ]:
train_matrix = np.load(open('./movies/train_e.npy', "rb"))
test_matrix = np.load(open('./movies/test_e.npy', "rb"))
labels = np.zeros(25000, dtype=np.int)
labels[:12500] = 1

### Inhibited softmax

In [ ]:
is_, optimizer, loss_function = create_model(
    ISMovie,
    loss_function=torch.nn.CrossEntropyLoss,
    optimizer=lambda x: optim.RMSprop(x, lr=0.0005)
)

def is_loss(model):

    return (
        lambda pred,aft_cauchy,y: loss_function(pred,y)
        - 0.000001 * aft_cauchy.sum()
    )

perform_training_sentiment(
    60,
    is_,
    train_matrix,
    test_matrix,
    labels,
    optimizer,
    is_loss(is_),
    100,
    'models/sentiment/is.torch',
    100,
    bce=False
)


# is_ = load_model(ISMovie, 'models/sentiment/is.torch')

In [ ]:
test_preds, test_labels, test_probs = test_eval(
    is_,
    generator_out_of_matrix(test_matrix, labels, 100, shuffle=False),
    is_sentiment=True
)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
accuracy_score(test_labels, np.argmax(test_probs, axis=1))

In [ ]:
log_loss(test_labels, softmax2d(test_probs[:,:2])[:,1])

### Second experiment

In [ ]:
roc, ac, fpr, tpr, pr, re = correlation_test_error_uncertainty(
    lambda x: softmax2d(x)[:,2],
    test_probs,
    test_labels,
    num_classes=2
)

roc

In [ ]:
ac

### Third experiment

In [ ]:
crd_matrix = np.load(open('./nonmovies/crd_e.npy', "rb"))
rp_matrix = np.load(open('./nonmovies/rp_e.npy', "rb"))
rp_preds, rp_labels, rp_probs = test_eval(
    is_,
    generator_out_of_matrix(rp_matrix, np.zeros(rp_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    is_sentiment=True
)
rp_labels = np.zeros(2000, dtype=np.int)
rp_labels[:1000] = 1
accuracy_score(rp_labels, rp_preds > 0.5)

In [ ]:
log_loss(rp_labels, softmax2d(rp_probs[:,:2])[:,1])

In [ ]:
crd_preds, crd_labels, crd_probs = test_eval(
    is_,
    generator_out_of_matrix(crd_matrix, np.zeros(crd_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    is_sentiment=True
)
roc, ac, fpr, tpr, pr, re = non_distribution(
    test_probs,
    softmax2d(test_probs)[:,2],
    softmax2d(crd_probs)[:,2],
    25315,
    25000
)

roc

In [ ]:
ac

In [ ]:
reuters_matrix = np.load(open('./nonmovies/reuters_e.npy', "rb"))
news_matrix = np.load(open('./nonmovies/news_e.npy', "rb"))
reuters_preds, reuters_labels, reuters_probs = test_eval(
    is_,
    generator_out_of_matrix(reuters_matrix, np.zeros(reuters_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    is_sentiment=True
)
roc, ac, fpr, tpr, pr, re = non_distribution(
    test_probs,
    softmax2d(test_probs)[:,2],
    softmax2d(reuters_probs)[:,2],
    46578,
    25000
)

roc

In [ ]:
ac

In [ ]:
news_preds, news_labels, news_probs = test_eval(
    is_,
    generator_out_of_matrix(news_matrix, np.zeros(news_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    is_sentiment=True
)
roc, ac, fpr, tpr, pr, re = non_distribution(
    test_probs,
    softmax2d(test_probs)[:,2],
    softmax2d(news_probs)[:,2],
    43828,
    25000
)

roc

In [ ]:
ac